this set up was made with the help of this guide https://nbviewer.org/github/microsoft/AIforEarthDataSets/blob/main/data/sentinel-5p.ipynb#Auth-files

# Environment setup

In [3]:
import os
import fsspec
import json
import urllib3
import xarray as xr
import numpy as np
from azure.storage.blob import ContainerClient
from datetime import datetime, timezone

# Not used directly, but needs to be installed to read NetCDF files with xarray
import h5netcdf

ModuleNotFoundError: No module named 'azure'

# Constants

In [ ]:
# Let's look at ozone concentration from mid-day on Jan 1, 2021
product = 'L2__O3____'
date = '2021/01/01'

# get Token from URL

In [ ]:
def get_Token_Data():
    http = urllib3.PoolManager()
    response = http.request('GET', 'https://planetarycomputer.microsoft.com/api/sas/v1/token/sentinel5euwest/sentinel-5p')
    data = json.loads(response.data.decode('utf-8'))
    return data

# Check Token File

## helper functions

In [ ]:
def is_file_empty(file_path):
    # check if file exist and is empty
    return os.path.exists(file_path) and os.stat(file_path).st_size == 0

def getTime():
    dt = datetime().now().astimezone(timezone.pst)
    dt_string = dt.isoformat(timespec = 'milliseconds').replace('+00:00', 'Z')
    return dt_string
    
def checkToken(jsonData):
    token_expiration = jsonData['msft:expiry']
    currTime = getTime()
    if token_expiration <= currTime :
        print('token expired! expiration Time: ', currTime)
        return false
    else:
        print('token still Valid! expiration Time ', curTime)
        return true

## Read Token

In [ ]:
sas_path = './tokens/sentinel-5p_sas.json'
is_empty = is_file_empty(sas_path)

if !is_empty:
    with open(sas_path, 'r+') as f:
        data = json.load(f)
        if checkToken(data):
            sas_token=data['token']
        else:
            newData = get_Token_Data()
            sas_token = newData['token']
            f.seek(0)
            json.dump(newData, f)
else:
    newData = get_Token_Data()
    sas_token = newData['token']
    f.seek(0)
    json.dump(newData, f)

# Azure storage constants

In [ ]:
storage_account_name = 'sentinel5euwest'
container_name = 'sentinel-5p'
storage_account_url = 'https://' + storage_account_name + '.blob.core.windows.net/'

container_client = ContainerClient(account_url=storage_account_url, 
                                                container_name=container_name,
                                                credential=sas_token)

# List products matching our product/date

In [ ]:
prefix = '/'.join(['TROPOMI',product,date])
print('Searching for prefix {}'.format(prefix))
generator = container_client.list_blobs(name_starts_with=prefix)
scene_paths = [blob.name for blob in generator]
print('\nFound {} matching scenes:\n'.format(len(scene_paths)))
for s in scene_paths:
    print(s.split('/')[-1])